# 📘 21_context_managers.ipynb

### 🧩 Topic: Context Managers in Python — `with`, `__enter__`, `__exit__`, and `contextlib`


## 🧠 1. What is a Context Manager?

A **context manager** is a Python object that sets up a runtime context and cleans up when leaving that context. The `with` statement uses context managers to ensure resources are properly acquired and released (files, locks, DB connections, etc.).

A context manager implements two methods:
- `__enter__(self)` — run at the start of the `with` block; its return value (if any) is bound to the `as` target.
- `__exit__(self, exc_type, exc, tb)` — run at the end of the `with` block; can suppress exceptions by returning `True`.


## 🔄 2. Visual Flow — `with` / Context Manager

```
with ContextManager() as resource:
    # __enter__ runs -> resource available
    do_work(resource)
# on exit: __exit__ runs (cleanup)
```

Flow diagram (ASCII):

```
enter context -> execute block -> exit context (cleanup)
```

## ⚙️ 3. Simple Example — Using `with` for Files

In [ ]:
# Recommended pattern for file handling
with open("/mnt/data/context_demo.txt", "w") as f:
    f.write("This file was written inside a with-block.\n")

# File is automatically closed after the block
print("File written and closed.")

## 🧱 4. Implementing a Class-based Context Manager

In [ ]:
class SimpleTimer:
    def __init__(self, name=None):
        from time import perf_counter
        self._start = None
        self.name = name or "Timer"
        self.perf_counter = perf_counter

    def __enter__(self):
        self._start = self.perf_counter()
        print(f"[{self.name}] started")
        return self  # returned object assigned to `as` target

    def __exit__(self, exc_type, exc, tb):
        duration = self.perf_counter() - self._start
        print(f"[{self.name}] finished: {duration:.6f}s")
        # Do not suppress exceptions; return False or None

# Demo
import time
with SimpleTimer("demo"):
    time.sleep(0.1)

## 🔁 5. How `__exit__` parameters work

`__exit__(exc_type, exc, tb)` receives exception info if an exception occurs inside the block — you can handle or suppress the exception here.

Example below demonstrates catching and suppressing an exception inside `__exit__`.  **Use suppression carefully**.

In [ ]:
class SuppressZeroDivision:
    def __enter__(self):
        print("Entering: will suppress ZeroDivisionError")
        return None
    def __exit__(self, exc_type, exc, tb):
        if exc_type is ZeroDivisionError:
            print("ZeroDivisionError suppressed by context manager")
            return True  # suppress the exception
        return False  # don't suppress other exceptions

with SuppressZeroDivision():
    1/0
print("Execution continues after suppressed exception.")

## 🧰 6. `contextlib` — Function-based Context Managers

In [ ]:
from contextlib import contextmanager

@contextmanager
def open_rw(path):
    f = open(path, "w+")
    try:
        yield f
    finally:
        f.close()

# Demo
with open_rw("/mnt/data/contextlib_demo.txt") as f:
    f.write("written via contextlib context manager\n")

## 🌍 7. Real-World Examples

In [ ]:
# Example: Database connection simulation
class FakeDBConnection:
    def __init__(self, name):
        self.name = name
        self.connected = False
    def connect(self):
        print(f"Connecting to {self.name}...")
        self.connected = True
    def close(self):
        print("Closing connection")
        self.connected = False

class DBContext:
    def __init__(self, db_name):
        self.db = FakeDBConnection(db_name)
    def __enter__(self):
        self.db.connect()
        return self.db
    def __exit__(self, exc_type, exc, tb):
        self.db.close()
        if exc_type:
            print("An error occurred inside DB context:", exc_type)
        # not suppressing exceptions

# Demo usage
with DBContext("my-db") as conn:
    print("Using connection:", conn.connected)

## 🧩 8. Mini Project — Resource Monitor Context Manager

Build a context manager that logs start/end time and usage info to a file. This helps monitor resource usage of code blocks.

In [ ]:
from datetime import datetime
class ResourceMonitor:
    def __init__(self, name, logfile="/mnt/data/resource_monitor.log"):
        self.name = name
        self.logfile = logfile

    def __enter__(self):
        self.start = datetime.now()
        with open(self.logfile, "a") as f:
            f.write(f"[{self.start}] START {self.name}\n")
        return self

    def __exit__(self, exc_type, exc, tb):
        self.end = datetime.now()
        duration = (self.end - self.start).total_seconds()
        with open(self.logfile, "a") as f:
            f.write(f"[{self.end}] END {self.name} duration={duration}s\n")
        if exc_type:
            with open(self.logfile, "a") as f:
                f.write(f"[{self.end}] ERROR {exc_type}: {exc}\n")
        # don't suppress exceptions
        return False

# Demo
with ResourceMonitor("heavy_task"):
    total = 0
    for i in range(100000):
        total += i*i
print("Resource monitor log appended.")

## 🔧 9. Using `contextlib.ExitStack` for dynamic context managers

In [ ]:
from contextlib import ExitStack

paths = ["/mnt/data/sample.txt", "/mnt/data/context_demo.txt"]
with ExitStack() as stack:
    files = [stack.enter_context(open(p, "r")) for p in paths if Path(p).exists()]
    for f in files:
        print("Read first line:", f.readline().strip())

## ⚠️ 10. Best Practices & Tips

- Prefer `with` for resource handling (files, locks, DB connections).  
- Avoid suppressing exceptions unless you explicitly handle them.  
- Use `contextlib.contextmanager` for simple use-cases; write class-based managers when you need state.  
- Use `ExitStack` to manage a dynamic number of context managers.  


## 💡 11. Beginner-Level Challenges

1️⃣ Write a context manager that temporarily changes the working directory and restores it on exit.  
2️⃣ Use `contextlib` to create a timer context manager that yields elapsed time in `__exit__`.  


In [ ]:
# 1️⃣ Temporary working directory context manager (example solution)
import os

class temp_cwd:
    def __init__(self, path):
        self.new_path = path
        self.old_path = None
    def __enter__(self):
        self.old_path = os.getcwd()
        os.chdir(self.new_path)
        return self.new_path
    def __exit__(self, exc_type, exc, tb):
        os.chdir(self.old_path)
        return False  # don't suppress exceptions

# Demo (ensure path exists)
Path("/mnt/data").mkdir(parents=True, exist_ok=True)
with temp_cwd("/mnt/data"):
    print("Inside temporary cwd:", os.getcwd())
print("Restored cwd:", os.getcwd())

## 💪 12. Advanced Challenges

1️⃣ Implement a context manager that acquires a file lock (simulate using a lock file) and releases it on exit.  
2️⃣ Create a context manager that measures memory usage of a block (use `tracemalloc` or `psutil`).  


## 🧠 13. Summary

| Concept | Notes |
|---|---|
| Context manager | Manages setup and teardown of resources |
| `__enter__` | Called at start of `with` block |
| `__exit__` | Called on exit; can handle exceptions |
| `contextlib.contextmanager` | Create context managers from generator functions |
| `ExitStack` | Manage dynamic set of context managers |


---
## ✅ Next Notebook
👉 `22_regular_expressions.ipynb` — Learn `re` module, pattern matching, groups, and common validation tasks.
